# Importing all required packages

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)

In [2]:
#reading the MNIST data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
import math
def batches(batch_size, features, labels):
    assert len(features) == len(labels)
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches


# Network Without any hidden layers

In [22]:
#building a simple logistic regression using tensorflow
n_features = len(mnist.train.images[0])
n_labels = len(mnist.train.labels[0])
train_features = mnist.train.images
train_labels = mnist.train.labels.astype(np.float32)
validation_features = mnist.
test_features = mnist.test.images
test_labels = mnist.test.labels.astype(np.float32)
#for feeding data into neural network we use placeholder
features = tf.placeholder(tf.float32,shape=[None,n_features])
y = tf.placeholder(tf.float32,shape=[None,n_labels])
#Weights are variables which are learnt during training of a neural network
w = tf.Variable(tf.truncated_normal((n_features,n_labels)))
b = tf.Variable(tf.zeros(n_labels))

learning_rate = 0.005
#with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #output = wx+b
    y_ = tf.add(tf.matmul(features,w),b)
    #for calculating error cross entropy
    #cross_entropy = tf.reduce_sum(tf.multiply(y,tf.log(y_)),reduction_indices=1)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y)
    loss = tf.reduce_mean(cross_entropy)    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    
    prediction = tf.nn.softmax(y_)
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    for epoch in range(300):
        for f,l in batches(128,train_features,train_labels):
            sess.run([optimizer],feed_dict={features:f,y:l})
        test_accuracy = sess.run(accuracy, feed_dict={features: test_features, y: test_labels})
        if epoch%50==0:
            print("epoch: ",epoch,"accuracy: ",test_accuracy)

epoch:  0 accuracy:  0.1963
epoch:  50 accuracy:  0.8209
epoch:  100 accuracy:  0.8577
epoch:  150 accuracy:  0.8709
epoch:  200 accuracy:  0.8799
epoch:  250 accuracy:  0.8854


In [23]:
test_accuracy

0.89000005

# Network With one hidden layer

In [24]:
n_hunits = 256
wh = tf.Variable(tf.truncated_normal((n_features,n_hunits)))
bh = tf.Variable(tf.zeros(n_hunits))
h_layer = tf.add(tf.matmul(features,wh),bh)
h_layer = tf.nn.relu(h_layer)

wo = tf.Variable(tf.truncated_normal((n_hunits,n_labels)))
bo = tf.Variable(tf.zeros(n_labels))

#output = wx+b
y_ = tf.add(tf.matmul(h_layer,wo),bo)

learning_rate = 0.005
#with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #for calculating error cross entropy
    #cross_entropy = tf.reduce_sum(tf.multiply(y,tf.log(y_)),reduction_indices=1)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y)
    loss = tf.reduce_mean(cross_entropy)    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    
    prediction = tf.nn.softmax(y_)
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    for epoch in range(300):
        for f,l in batches(128,train_features,train_labels):
            sess.run([optimizer],feed_dict={features:f,y:l})
        test_accuracy = sess.run(accuracy, feed_dict={features: test_features, y: test_labels})
        if epoch%50==0:
            print("epoch: ",epoch,"accuracy: ",test_accuracy)

epoch:  0 accuracy:  0.7261
epoch:  50 accuracy:  0.9157
epoch:  100 accuracy:  0.9263
epoch:  150 accuracy:  0.9303
epoch:  200 accuracy:  0.9344
epoch:  250 accuracy:  0.9365
